# **Web Server Log Analysis with Apache Spark**
 


### How to complete this assignment
 
#### This assignment is broken up into sections with bite-sized examples for demonstrating Spark functionality for log processing. For each problem, you should start by thinking about the algorithm that you will use to *efficiently* process the log in a parallel, distributed manner. This means using the various [RDD](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) operations along with [`lambda` functions](https://docs.python.org/2/tutorial/controlflow.html#lambda-expressions) that are applied at each worker.
 
### This assignment consists of 4 parts:
#### *Part 1*: Apache Web Server Log file format: *parital Solution Given*
#### *Part 2*: Sample Analyses on the Web Server Log File: *Full Solution Given*
#### *Part 3*: Analyzing Web Server Log File
#### *Part 4*: Exploring 404 Response Codes

#### Replace `<Fill in>` with appropriate code

### **Part 1: Apache Web Server Log file format**
#### The log files that we use for this assignment are in the [Apache Common Log Format (CLF)](http://httpd.apache.org/docs/1.3/logs.html#common). The log file entries produced in CLF will look something like this:
`127.0.0.1 - - [01/Aug/1995:00:00:01 -0400] "GET /images/launch-logo.gif HTTP/1.0" 200 1839`
 
#### Each part of this log entry is described below.
* `127.0.0.1`
#### This is the IP address (or host name, if available) of the client (remote host) which made the request to the server.
 
* `-`
#### The "hyphen" in the output indicates that the requested piece of information (user identity from remote machine) is not available.
 
* `-`
#### The "hyphen" in the output indicates that the requested piece of information (user identity from local logon) is not available.
 
* `[01/Aug/1995:00:00:01 -0400]`
#### The time that the server finished processing the request. The format is:
`[day/month/year:hour:minute:second timezone]`
  * day = 2 digits
  * month = 3 letters
  * year = 4 digits
  * hour = 2 digits
  * minute = 2 digits
  * second = 2 digits
  * zone = (\+ | \-) 4 digits
 
* `"GET /images/launch-logo.gif HTTP/1.0"`
#### This is the first line of the request string from the client. It consists of a three components: the request method (e.g., `GET`, `POST`, etc.), the endpoint (a [Uniform Resource Identifier](http://en.wikipedia.org/wiki/Uniform_resource_identifier)), and the client protocol version.
 
* `200`
#### This is the status code that the server sends back to the client. This information is very valuable, because it reveals whether the request resulted in a successful response (codes beginning in 2), a redirection (codes beginning in 3), an error caused by the client (codes beginning in 4), or an error in the server (codes beginning in 5). The full list of possible status codes can be found in the HTTP specification ([RFC 2616](https://www.ietf.org/rfc/rfc2616.txt) section 10).
 
* `1839`
#### The last entry indicates the size of the object returned to the client, not including the response headers. If no content was returned to the client, this value will be "-" (or sometimes 0).
 
#### Note that log files contain information supplied directly by the client, without escaping. Therefore, it is possible for malicious clients to insert control-characters in the log files, *so care must be taken in dealing with raw logs.*
 
### NASA-HTTP Web Server Log
#### For this assignment, we will use a data set from NASA Kennedy Space Center WWW server in Florida. The full data set is freely available (http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html) and contains two month's of all HTTP requests. We are using a subset that only contains several days worth of requests.

### **(1a) Data Prepration**
#### Using the CLF as defined above, we create a regular expression pattern to extract the nine fields of the log line using the Python regular expression [`search` function](https://docs.python.org/2/library/re.html#regular-expression-objects). The function returns a pair consisting of a Row object and 1. If the log line fails to match the regular expression, the function returns a pair consisting of the log line string and 0. A '-' value in the content size field is cleaned up by substituting it with 0. The function converts the log line's date string into a Python `datetime` object using the given `parse_apache_time` function.

In [1]:
import pyspark
#sc = pyspark.SparkContext('local[*]')
# Display the type of the Spark Context sc
#type(sc)
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark ML Class Assisgnment") \
    .getOrCreate()
sc=spark.sparkContext

In [2]:
import re
import datetime

from pyspark.sql import Row

month_map = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
    'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12}

def parse_apache_time(s):
    """ Convert Apache time format into a Python datetime object
    Args:
        s (str): date and time in Apache time format
    Returns:
        datetime: datetime object (ignore timezone for now)
    """
    return datetime.datetime(int(s[7:11]),
                             month_map[s[3:6]],
                             int(s[0:2]),
                             int(s[12:14]),
                             int(s[15:17]),
                             int(s[18:20]))


def parseApacheLogLine(logline):
    """ Parse a line in the Apache Common Log format
    Args:
        logline (str): a line of text in the Apache Common Log format
    Returns:
        tuple: either a dictionary containing the parts of the Apache Access Log and 1,
               or the original invalid log line and 0
    """
    match = re.search(APACHE_ACCESS_LOG_PATTERN, logline)
    if match is None:
        return (logline, 0)
    size_field = match.group(9)
    if size_field == '-':
        size = int(0)
    else:
        size = int(match.group(9))
    return (Row(
        host          = match.group(1),
        client_identd = match.group(2),
        user_id       = match.group(3),
        date_time     = parse_apache_time(match.group(4)),
        method        = match.group(5),
        endpoint      = match.group(6),
        protocol      = match.group(7),
        response_code = int(match.group(8)),
        content_size  = size
    ), 1)

In [3]:
# A regular expression pattern to extract fields from the log line
APACHE_ACCESS_LOG_PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S*)" (\d{3}) (\S+)'

### **(1b) Configuration and Initial RDD Creation**
#### You are ready to specify the input log file and create an RDD containing the parsed log file data. The log file has already been provided to you.
 
#### To create the primary RDD that you will use in the rest of this assignment, first load the text file using [`sc.textfile(logFile)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.textFile) to convert each line of the file into an element in an RDD.
#### Next, use Map fucntion (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) to apply the parse function to each element (that is, a line from the log file) in the RDD and turn each line into a pair [`Row` object](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Row).
#### Finally, cache the RDD in memory since you will use it throughout this notebook.

In [6]:
logFile = "access_log_Aug95"
parsed_logs = sc.textFile(logFile).map(parseApacheLogLine).cache()



In [7]:
#How many lines the file has?
parsed_logs.count()

999999

In [9]:

# create access log by filtering those rows which has 
# 1 in the second element of the tuple and cache it
access_logs = parsed_logs.filter(lambda s: s[1] == 1).cache()
# create failed logs by filtering those rows which has 0 in the second element of the tuple 


In [11]:

failed_logs = parsed_logs.filter(lambda s: s[1] == 0).cache()

#print how many lines failed.
print(access_logs.count())
    

999826


In [12]:
print(failed_logs.count())

173


In [32]:
## failed_logs = parsed_logs.filter(lambda s: s[1] == 0).map( lambda s: s[0]).cache()
## if we do this we would not need to change log[0] on the following lines

In [13]:
parsed_logs.take(2)

[(Row(host='in24.inetnebr.com', client_identd='-', user_id='-', date_time=datetime.datetime(1995, 8, 1, 0, 0, 1), method='GET', endpoint='/shuttle/missions/sts-68/news/sts-68-mcc-05.txt', protocol='HTTP/1.0', response_code=200, content_size=1839),
  1),
 (Row(host='uplherc.upl.com', client_identd='-', user_id='-', date_time=datetime.datetime(1995, 8, 1, 0, 0, 7), method='GET', endpoint='/', protocol='HTTP/1.0', response_code=304, content_size=0),
  1)]

### **Part 2: Sample Analyses on the Web Server Log File**
 
#### Now that you have an RDD containing the log file as a set of Row objects, you can perform various analyses.
 
#### **(2a) Example: Content Size Statistics**
 
#### Compute some statistics about the sizes of content being returned by the web server. In particular, we'd like to know what are the average, minimum, and maximum content sizes.
 
#### You can compute the statistics by applying a `map` to the `access_logs` RDD. The `lambda` function you want for the map should extract the `content_size` field from the RDD. The map produces a new RDD containing only the `content_sizes` (one element for each Row object in the `access_logs` RDD). To compute the minimum and maximum statistics, you can use [`min()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.min) and [`max()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.max) functions on the new RDD. You can compute the average statistic by using the [`reduce`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduce) function with a `lambda` function that sums the two inputs, which represent two elements from the new RDD that are being reduced together. The result of the `reduce()` is the total content size from the log and it is to be divided by the number of requests as determined using the [`count()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.count) function on the new RDD.

In [14]:
# Calculate statistics based on the content size.
content_sizes = access_logs.map(lambda log: log.content_size).cache()
print ('Content Size Avg: %i, Min: %i, Max: %s' % (
    content_sizes.reduce(lambda a, b : a + b) / content_sizes.count(),
    content_sizes.min(),
    content_sizes.max()))

Content Size Avg: 17578, Min: 0, Max: 3421948


#### **(2b) Example: Response Code Analysis**
#### Look at the response codes that appear in the log. As with the content size analysis, first create a new RDD by using a `lambda` function to extract the `response_code` field from the `access_logs` RDD. The difference here is that you will use a [pair tuple](https://docs.python.org/2/tutorial/datastructures.html?highlight=tuple#tuples-and-sequences) instead of just the field itself. Using a pair tuple consisting of the response code and 1 will let you count how many records  a particular response code have. Using the new RDD, perform a [`reduceByKey`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) function. Use the simple `lambda` function of adding the two values. Then, cache the resulting RDD and create a list by using the [`take`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take) function.

In [15]:
# Response Code to Count
responseCodeToCount = (access_logs
                       .map(lambda log: (log[0].response_code, 1))
                       .reduceByKey(lambda a, b : a + b)
                       .cache())
responseCodeToCountList = responseCodeToCount.take(100)
print ('Found %d response codes' % len(responseCodeToCountList))
print ('Response Code Counts: %s' % responseCodeToCountList)
assert len(responseCodeToCountList) == 7


Found 7 response codes
Response Code Counts: [(200, 901680), (304, 76577), (404, 5952), (500, 2), (501, 17), (302, 15546), (403, 52)]


#### **(2c) Example: Frequent Hosts**
#### Now, look at hosts that have accessed the server multiple times (e.g., more than ten times). As with the response code analysis in (2b), first create a new RDD by using a `lambda` function to extract the `host` field from the `access_logs` RDD using a pair tuple consisting of the host and 1 which will let you count how many records were created by a particular host's request. Using the new RDD, perform a `reduceByKey` function with a `lambda` function that adds the two values. Then filter the result based on the count of accesses by each host (the second element of each pair) being greater than ten. Next, extract the host name by performing a `map` with a `lambda` function that returns the first element of each pair. Finally, extract 20 elements from the resulting RDD - *note that the choice of which elements are returned is not guaranteed to be deterministic.*

In [16]:
# Any hosts that has accessed the server more than 10 times.
hostCountPairTuple = access_logs.map(lambda log: (log[0].host, 1))

hostSum = hostCountPairTuple.reduceByKey(lambda a, b : a + b)

hostMoreThan10 = hostSum.filter(lambda s: s[1] > 10)

hostsPick20 = (hostMoreThan10
               .map(lambda s: s[0])
               .take(20))

print ('Any 20 hosts that have accessed more then 10 times: %s' % hostsPick20)
# An example: [u'204.120.34.185', u'204.243.249.9', u'slip1-32.acs.ohio-state.edu', u'lapdog-14.baylor.edu', u'199.77.67.3', u'gs1.cs.ttu.edu', u'haskell.limbex.com', u'alfred.uib.no', u'146.129.66.31', u'manaus.bologna.maraut.it', u'dialup98-110.swipnet.se', u'slip-ppp02.feldspar.com', u'ad03-053.compuserve.com', u'srawlin.opsys.nwa.com', u'199.202.200.52', u'ix-den7-23.ix.netcom.com', u'151.99.247.114', u'w20-575-104.mit.edu', u'205.25.227.20', u'ns.rmc.com']

Any 20 hosts that have accessed more then 10 times: ['www-c3.proxy.aol.com', 'www-d3.proxy.aol.com', 'www-d4.proxy.aol.com', 'piweba1y.prodigy.com', 'pm9.j51.com', 'pipe1.nyc.pipeline.com', 'dialip129.gov.bc.ca', 'palpk-s13.intac.com', 'www-d1.proxy.aol.com', 'tia1.eskimo.com', 'remote.redzone.com', 'www-c1.proxy.aol.com', 'pme609.onramp.awinc.com', 'prakinf2.prakinf.tu-ilmenau.de', '168.78.14.166', '199.190.95.40', 'bonnou.lab.kdd.co.jp', 'gatekeeper.us.oracle.com', 'www-a1.proxy.aol.com', 'kingston-ts-16.nstn.ca']


#### **(2d) Example: Top Endpoints**
#### For the final example, we'll look at the top endpoints (URIs) in the log. To determine them, we first create a new RDD by using a `lambda` function to extract the `endpoint` field from the `access_logs` RDD using a pair tuple consisting of the endpoint and 1 which will let us count how many records were created by a particular host's request. Using the new RDD, we perform a `reduceByKey` function with a `lambda` function that adds the two values. We then extract the top ten endpoints by performing a [`takeOrdered`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered) with a value of 10 and a `lambda` function that multiplies the count (the second element of each pair) by -1 to create a sorted list with the top endpoints at the bottom.

In [17]:
# Top Endpoints
endpointCounts = (access_logs
                  .map(lambda log: (log[0].endpoint, 1))
                  .reduceByKey(lambda a, b : a + b))

topEndpoints = endpointCounts.takeOrdered(10, lambda s: -1 * s[1])

print ('Top Ten Endpoints: %s' % topEndpoints)

Top Ten Endpoints: [('/images/NASA-logosmall.gif', 56967), ('/images/KSC-logosmall.gif', 48242), ('/images/MOSAIC-logosmall.gif', 41929), ('/images/USA-logosmall.gif', 41714), ('/images/WORLD-logosmall.gif', 41354), ('/images/ksclogo-medium.gif', 39498), ('/ksc.html', 27265), ('/history/apollo/images/apollo-logo1.gif', 25567), ('/images/launch-logo.gif', 23570), ('/', 19486)]


### **Part 3: Analyzing Web Server Log File**
 
#### Now it is your turn to perform analyses on web server log files.

#### **(3a) Exercise: Top Ten Error Endpoints**
#### What are the top ten endpoints which did not have return code 200? Create a sorted list containing top ten endpoints and the number of times that they were accessed with non-200 return code.
 
#### Think about the steps that you need to perform to determine which endpoints did not have a 200 return code, how you will uniquely count those endpoints, and sort the list.

In [34]:
# TODO: Replace <FILL IN> with appropriate code
# HINT: Each of these <FILL IN> below could be completed with a single transformation or action.
# You are welcome to structure your solution in a different way, so long as
# you ensure the variables used in the next Test section are defined (ie. endpointSum, topTenErrURLs).

not200 = access_logs.filter(lambda log: log[0].response_code != 200) #. cache()



In [35]:
endpointCountPairTuple = not200.map(lambda log: (log[0].endpoint, 1))


In [36]:
endpointSum = endpointCountPairTuple.reduceByKey(lambda a, b : a + b)

In [37]:
endpointCountPairTuple.take(2)

[('/', 1), ('/images/ksclogo-medium.gif', 1)]

In [38]:
topTenErrURLs = endpointSum.takeOrdered(10, lambda s: -1 * s[1])
print('Top Ten failed URLs: %s' % topTenErrURLs)

Top Ten failed URLs: [('/images/NASA-logosmall.gif', 8377), ('/images/KSC-logosmall.gif', 6944), ('/images/MOSAIC-logosmall.gif', 4967), ('/images/USA-logosmall.gif', 4926), ('/images/WORLD-logosmall.gif', 4800), ('/images/ksclogo-medium.gif', 4538), ('/history/apollo/images/apollo-logo1.gif', 2764), ('/images/launch-logo.gif', 2685), ('/', 2151), ('/images/ksclogosmall.gif', 1555)]


#### **(3b) Exercise: Number of Unique Hosts**
#### How many unique hosts are there in the entire log?
 
#### Think about the steps that you need to perform to count the number of different hosts in the log.

In [40]:
# TODO: Replace <FILL IN> with appropriate code
# HINT: Do you recall the tips from (3a)? Each of these <FILL IN> could be a transformation or action.

hosts = access_logs.map(lambda log: (log[0].host, 1))

uniqueHosts = hosts.reduceByKey(lambda a, b : a+b)
## Could have distinct()
uniqueHostCount = uniqueHosts.count()
print ('Unique hosts: %d' % uniqueHostCount)

Unique hosts: 52642


#### **(3c) Exercise: Number of Unique Daily Hosts**
#### Determine the number of unique hosts in the entire log on a day-by-day basis. This computation will give counts of the number of unique daily hosts. We'd like a list sorted by increasing day of the month which includes the day of the month and the associated number of unique hosts for that day. Make sure you cache the resulting RDD `dailyHosts` so that you can reuse it in the next exercise.
 
#### Think about the steps that you need to perform to count the number of different hosts that make requests *each* day.
#### *Since the log only covers a single month, you can ignore the month.*

#### Note: These analysis may take long time due to small cluster we have

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

dayToHostPairTuple = access_logs.map(lambda log: (log[0].date_time.day, log[0].host))

dayGroupedHosts = dayToHostPairTuple.combineByKey(lambda v: [v], lambda c, v: c+ [v], lambda c1, c2: c1+c2)

dayHostCount = dayGroupedHosts.map(lambda kv: (kv[0], len(set(kv[1])) ) )
                                                   
dailyHosts = dayHostCount.sortByKey(ascending = True).cache()

dailyHostsList = dailyHosts.take(30)
print ('Unique hosts per day: %s' % dailyHostsList)

#### **(3d) Exercise: Average Number of Daily Requests per Hosts**
#### Determine the average number of requests on a day-by-day basis. We'd like a list by increasing day of the month and the associated average number of requests per host for that day. Make sure you cache the resulting RDD `avgDailyReqPerHost` so that you can reuse it in the next exercise.
#### To compute the average number of requests per host, get the total number of request across all hosts and divide that by the number of unique hosts.
#### *Since the log only covers a single month, you can skip checking for the month.*
#### *Also to keep it simple, when calculating the approximate average use the integer value - you do not need to upcast to float*

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

dayAndHostTuple = access_logs.<FILL IN>

groupedByDay = dayAndHostTuple.<FILL IN>

sortedByDay = groupedByDay.<FILL IN>

avgDailyReqPerHost = sortedByDay.map(lambda kv: (kv[0], len(kv[1]) / len(set(kv[1])))).cache()

avgDailyReqPerHostList = avgDailyReqPerHost.take(30)
print ('Average number of daily requests per Hosts is %s' % avgDailyReqPerHostList)

### **Part 4: Exploring 404 Response Codes**
 
#### 404 errors are returned when an endpoint is not found by the server (i.e., a missing page or object).

#### **(4a) Exercise: Counting 404 Response Codes**
#### Create a RDD containing only log records with a 404 response code. Make sure you `cache()` the RDD `badRecords` as you will use it in the rest of this exercise.
 
#### How many 404 records are in the log?

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

badRecords = (access_logs
              .<FILL IN>
              .cache())
print ('Found %d 404 URLs' % badRecords.count())

#### **(4b) Exercise: Listing 404 Response Code Records**
#### Using the RDD containing only log records with a 404 response code that you cached in part (4a), print out a list up to 40 **distinct** endpoints that generate 404 errors -  *no endpoint should appear more than once in your list.*

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

badEndpoints = badRecords.<FILL IN>

badUniqueEndpoints = badEndpoints.<FILL IN>

badUniqueEndpointsPick40 = badUniqueEndpoints.take(40)
print ('404 URLS: %s' % badUniqueEndpointsPick40)

#### **(4c) Exercise: Listing the Top Twenty 404 Response Code Endpoints**
####Using the RDD containing only log records with a 404 response code that you cached in part (4a), print out a list of the top twenty endpoints that generate the most 404 errors.
####*Remember, top endpoints should be in sorted order*

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

badEndpointsCountPairTuple = badRecords.<FILL IN>

badEndpointsSum = badEndpointsCountPairTuple.<FILL IN>

badEndpointsTop20 = badEndpointsSum.<FILL IN>
print ('Top Twenty 404 URLs: %s' % badEndpointsTop20)

#### **(4d) Exercise: Listing the Top Twenty-five 404 Response Code Hosts**
#### Instead of looking at the endpoints that generated 404 errors, look at the hosts that encountered 404 errors. Using the RDD containing only log records with a 404 response code that you cached in part (4a), print out a list of the top twenty-five hosts that generate the most 404 errors.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

errHostsCountPairTuple = badRecords.<FILL IN>

errHostsSum = errHostsCountPairTuple.<FILL IN>

errHostsTop25 = errHostsSum.<FILL IN>
print ('Top 25 hosts that generated errors: %s' % errHostsTop25)

#### **(4e) Exercise: Listing 404 Response Codes per Day**
####  Break down the 404 requests by day (`cache()` the RDD `errDateSorted`) and get the daily counts sorted by day as a list.
#### *Since the log only covers a single month, you can ignore the month in your checks.*

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

errDateCountPairTuple = badRecords.<FILL IN>

errDateSum = errDateCountPairTuple.<FILL IN>

errDateSorted = (errDateSum
                 .<FILL IN>
                 .cache())

errByDate = errDateSorted.collect()
print ('404 Errors by day: %s' % errByDate)

#### **(4f) Exercise: Top Five Days for 404 Response Codes **
#### Using the RDD `errDateSorted` you cached in the part (4e), what are the top five days for 404 response codes and the corresponding counts of 404 response codes?

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

topErrDate = errDateSorted.<FILL IN>
print ('Top Five dates for 404 requests: %s' % topErrDate)

#### **(4h) Exercise: Hourly 404 Response Codes**
#### Using the RDD `badRecords` you cached in the part (4a) and by hour of the day and in increasing order, create an RDD containing how many requests had a 404 return code for each hour of the day (midnight starts at 0). Cache the resulting RDD hourRecordsSorted and print that as a list.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

hourCountPairTuple = badRecords.<FILL IN>

hourRecordsSum = hourCountPairTuple.<FILL IN>

hourRecordsSorted = (hourRecordsSum
                     .<FILL IN>
                     .cache())

errHourList = hourRecordsSorted.collect()
print ('Top hours for 404 requests: %s' % errHourList)